In [34]:
import numpy as np
import ggseg
import pickle

from matplotlib import pyplot as plt 
import pandas as pd 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.offline as pyo

import json

import warnings
warnings.filterwarnings("ignore")

## Load pickle

In [35]:
# Import Pikle File
read_input_file = open('data/Data.pickle','rb')
load_inputs = pickle.load(read_input_file)
read_input_file.close()

T, S, SBoot = load_inputs

### Load mappings

In [36]:
f = open('data/ASEG_3D_combined.json')
map_aseg = json.load(f)
f.close()

f = open('/Users/macos/Downloads/DK_3D_combined.json')
map_dk = json.load(f)
f.close()

In [37]:
subtype_labels = ['Subcortical subtype', 'Frontal subtype', 'Parietal subtype','Typical subtype']

## DK-atlas

In [38]:
def dk_3D(T,S, mapped_dict, subtype_labels = None, subtype = None):
    
    """
    Creates a dictionary, which can be used as input to ggseg3d() function
    :param T: Timeline object
    :param S: subtyping dictionary, subtypes for each patient individually
    :param mapped_dict: dictionary from loaded JSON file, key: values --> T.biomarker_labels: list(DK-labels)
    :param subtype_labels: a list with names of the subtypes (optional)
    :param subtype: name or index of the subtype from subtype_lables (optional, choses first available subtype as default)  
    :return: dictionary with scores for each DK region for chosen subtype
    """
    
    unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
    if subtype_labels is None:
        subtype_labels = {f'Subtype {i}': i for i in range(len(unique_subtypes))}
        if subtype is None:
            subtype = next(iter(subtype_labels))
    else:
        subtype_labels = {subtype_labels[i]: i for i in range(len(unique_subtypes))}
    
    if subtype is None:
        subtype = subtype_labels[0]  
        
    # clean names from capital letters
    labels = list(map(lambda x: x.lower(), T.biomarker_labels))
    labels_dict = {num: label.lower() for num, label in enumerate(labels)}
    
    print(subtype_labels)
    # order the regions
    order = T.sequence_model['ordering'][subtype_labels[subtype]] 
    labels_ordered = []
    for o in order:
        labels_ordered.append(labels_dict[o])   
         
           
    dic = dict(zip(labels_ordered, T.sequence_model['event_centers'][subtype_labels[subtype]]))
                    
    # flat lost of dict values (single list of DK-labels)
    dk_flat = [x for v in mapped_dict.values() for x in v]
    
    # Separate hemispheres
    hemi = []
    for idx, region in enumerate(dk_flat):
        if 'left' in region:
            hemi.append('left')
            dk_flat[idx]=dk_flat[idx].replace(' left','')
        elif 'right' in region:
            hemi.append('right')
            dk_flat[idx]=dk_flat[idx].replace(' right','')
        else:
            hemi.append('subcort')
                
    #Match T.biomarker_labels to DK labels
    list_plot = list()
    for key in mapped_dict.keys():
        for item in mapped_dict[key]:
            list_plot.append(dic[key])
            
    # create DataFrame
    dic_dk = {'region': dk_flat, 'hemi':hemi, 'p': list_plot}
    df = pd.DataFrame(dic_dk)
    
    return df

## Save the output to use as input in ggseg_3d() in R

In [39]:
dk_new = dk_3D(T, S, mapped_dict = map_dk, 
               subtype = 'Subcortical subtype',
               subtype_labels=subtype_labels
              )
dk_new
# dk.to_csv('data/dk_R_Subtype 0.csv', index = False)

{'Subcortical subtype': 0, 'Frontal subtype': 1, 'Parietal subtype': 2, 'Typical subtype': 3}


,region,hemi,p
0,bankssts,left,0.472310
1,transverse temporal,left,0.472310
2,superior temporal,left,0.472310
3,temporal pole,left,0.472310
4,entorhinal,left,0.472310
...,...,...,...
63,isthmus cingulate,left,0.836612
64,posterior cingulate,right,0.836612
65,isthmus cingulate,right,0.836612
66,parahippocampal,left,0.711971


In [7]:
# pd.read_csv("data/dk_R_Subtype 3.csv") 

## Aseg atlas

In [40]:
def aseg_3D(T, S, mapped_dict, subtype_labels = None, subtype = None):
    
    """
    Creates a dictionary, which can be used as input to ggseg.plot_dk() function
    :param T: Timeline object
    :param S: subtyping dictionary, subtypes for each patient individually
    :param mapped_dict: dictionary from loaded JSON file, key: values --> T.biomarker_labels: list(DK-labels)
    :param subtype_labels: a list with names of the subtypes (optional)
    :param subtype: name or index of the subtype from subtype_lables (optional, choses first available subtype as default)  
    :return: dictionary with scores for each DK region for chosen subtype
    """

    unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
    if subtype_labels is None:
        subtype_labels = {f'Subtype {i}': i for i in range(len(unique_subtypes))}
        if subtype is None:
            subtype = next(iter(subtype_labels))
    else:
        subtype_labels = {subtype_labels[i]: i for i in range(len(unique_subtypes))}
    
    if subtype is None:
        subtype = subtype_labels[0] 
        
    # clean label names
    labels = T.biomarker_labels
    labels_dict = {num: label.lower() for num, label in enumerate(labels)}
    order = T.sequence_model['ordering'][subtype_labels[subtype]]
        
    # order the regions
    labels_ordered = []
    for o in order:
        labels_ordered.append(labels_dict[o])
        
    # Dictionary with all labels
    dic = dict(zip(labels_ordered, T.sequence_model['event_centers'][subtype_labels[subtype]]))
    
    # flat list of dict values (single list of DK-labels)
    aseg_flat = [x for v in mapped_dict.values() for x in v]
    
    #Match T.biomarker_labels to DK labels
    list_plot = list()
    for key in mapped_dict.keys():
        for item in mapped_dict[key]:
            list_plot.append(dic[key])
            
    # create DataFrame
    dic_aseg = {'region': aseg_flat, 'p':list_plot}     
    df = pd.DataFrame(dic_aseg)
        
    return df

## Save the output to use as input in ggseg_3d() in R

In [41]:
# NEW DATA
aseg_new = aseg_3D(T,S, map_aseg, 
               subtype = 'Subcortical subtype',
               subtype_labels=subtype_labels
                  )
aseg_new

# aseg.to_csv('data/aseg_R_Subtype 3.csv', index = False)

,region,p
0,Right-Thalamus-Proper,0.282890
1,Left-Thalamus-Proper,0.282890
2,Right-Caudate,0.016190
3,Left-Caudate,0.016190
4,Right-Putamen,0.138880
5,Left-Putamen,0.138880
6,Right-Hippocampus,0.369782
7,Left-Hippocampus,0.369782
8,Right-Amygdala,0.346440
9,Left-Amygdala,0.346440
